In [11]:
import os
import pandas as pd
import pickle
import biotype.similarity as sim

In [12]:
DATA_DIR_PATH = "datasets/export"
META_FILE_PATH = "datasets/meta.csv"
PEAK_DIR_PATH = "extracted_peaks"
files =  os.listdir(DATA_DIR_PATH)
files = [fn  for fn in files if fn.endswith(".txt")]
files = sorted(files)
for file in files[:5] :
    print(file)

L100_0_G7_1.txt
L100_0_G8_1.txt
L101_0_A1_1.txt
L101_0_A2_1.txt
L102_0_A3_1.txt


In [13]:
n = len(files)
all_peaks = []
for i in range(n):
    df = None
    df = pd.read_table(f"{DATA_DIR_PATH}/{files[i]}",sep=" ", header=None,names=['m/z', 'intensity']) 
    x, y = df['m/z'].to_numpy(), df['intensity'].to_numpy()

    pickle_file = files[i][:-4] # trim '.txt'
    with open(f'{PEAK_DIR_PATH}/{pickle_file}_peaks.pkl', 'rb') as peak_file:
        peaks = pickle.load(peak_file)
        # peaks_extracted +=  [(x[peaks], y[peaks])] 
        for p in peaks:
            all_peaks += [(x[p],y[p], i)]

all_peaks = sorted(all_peaks, key=lambda x: x[0])
print(all_peaks[:5])

[(1000.748, 5652, 42), (1000.748, 2899, 90), (1000.871, 7600, 93), (1000.871, 6380, 95), (1000.948, 2654, 100)]


In [14]:
import itertools

def pairwise(iterable,):
    "s -> (s0, s1), (s1, s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a,b)


In [16]:
def gap(peaks, a, b):
    """ find the position where the gap to next element is maximal   
    """
    max_gap, k = 0, 0
    for i in range(a, b):
        # print(all_peaks[i], '..', all_peaks[i+1])
        if abs(peaks[i][0]-peaks[i+1][0]) > max_gap:
            max_gap = abs(peaks[i][0]-peaks[i+1][0])
            k = i
            
    return k
    
    
def align(peaks, delta=3):
    div = [] 
    k = -1
    for a, b in pairwise(peaks):
        k = k + 1
        if abs(a[0] - b[0]) > delta:
            div +=[k]

    loop = True        
    while loop:
        new_div = div[:]
        loop = False
        for a, b in pairwise(div):
            if abs(peaks[a+1][0] - peaks[b][0]) > delta:
                k = gap(peaks, a+1, b)
                new_div += [k]
                loop = True
                
        div =  sorted(list(set(new_div)))
        
    return div


k = gap(all_peaks,168,244)
print("*" * 30)
print (k)
print(all_peaks[k])
print(all_peaks[k+1])

delta = 5
div = align(all_peaks, delta)
print(div[:10])

#[32, 106, 120, 164, 167, 321, 356, 366, 430, 533]

******************************
220
(1049.304, 25750, 30)
(1053.259, 7319, 43)
[32, 106, 120, 164, 167, 219, 220, 321, 356, 366]


In [33]:
for a, b in pairwise(div):
    diff = abs(all_peaks[a+1][0] - all_peaks[b][0]) 
    if diff > delta:
        print(f"{a+1},{b}, diff={diff:.2f}:  {all_peaks[a+1]}  <->  {all_peaks[b]}");

# print (len(all_peaks)-1) 

# for i in div:
#     print(i, all_peaks[i],all_peaks[i+1])

In [39]:
peak_id = 1
aligned_peaks = dict()
new_div = [-1] + div[:] + [len(all_peaks)-1]
for a, b in pairwise(div):
    for mz, intensity, ms in all_peaks[a+1:b]:
        aligned_peaks[(mz, ms)] = peak_id
    if a+1 < b:
        peak_id = peak_id + 1

for a in aligned_peaks:
    print(a, ': ', aligned_peaks[a])

(1007.214, 43) :  1
(1007.309, 20) :  1
(1007.405, 87) :  1
(1007.5, 21) :  1
(1007.5, 91) :  1
(1007.595, 99) :  1
(1007.716, 97) :  1
(1007.737, 24) :  1
(1007.832, 6) :  1
(1007.881, 42) :  1
(1007.881, 90) :  1
(1007.932, 0) :  1
(1008.023, 22) :  1
(1008.023, 57) :  1
(1008.023, 68) :  1
(1008.023, 75) :  1
(1008.023, 77) :  1
(1008.118, 14) :  1
(1008.118, 34) :  1
(1008.118, 41) :  1
(1008.118, 81) :  1
(1008.122, 1) :  1
(1008.213, 31) :  1
(1008.213, 47) :  1
(1008.213, 69) :  1
(1008.213, 82) :  1
(1008.213, 83) :  1
(1008.309, 5) :  1
(1008.309, 10) :  1
(1008.309, 23) :  1
(1008.309, 25) :  1
(1008.309, 37) :  1
(1008.309, 46) :  1
(1008.309, 48) :  1
(1008.309, 49) :  1
(1008.309, 59) :  1
(1008.309, 61) :  1
(1008.309, 72) :  1
(1008.309, 80) :  1
(1008.309, 88) :  1
(1008.404, 11) :  1
(1008.404, 28) :  1
(1008.404, 45) :  1
(1008.404, 56) :  1
(1008.404, 58) :  1
(1008.404, 63) :  1
(1008.404, 74) :  1
(1008.404, 76) :  1
(1008.404, 79) :  1
(1008.404, 85) :  1
(1008.47

(1165.182, 99) :  17
(1165.285, 21) :  17
(1165.285, 42) :  17
(1165.387, 20) :  17
(1165.387, 98) :  17
(1165.489, 65) :  17
(1165.489, 86) :  17
(1165.489, 87) :  17
(1165.592, 91) :  17
(1166.002, 47) :  17
(1166.002, 53) :  17
(1166.002, 58) :  17
(1166.049, 0) :  17
(1166.105, 34) :  17
(1166.105, 37) :  17
(1166.105, 46) :  17
(1166.207, 25) :  17
(1166.207, 61) :  17
(1166.207, 89) :  17
(1166.254, 1) :  17
(1166.265, 97) :  17
(1166.31, 71) :  17
(1166.31, 84) :  17
(1166.31, 85) :  17
(1166.367, 96) :  17
(1166.412, 11) :  17
(1166.412, 13) :  17
(1166.412, 35) :  17
(1166.412, 48) :  17
(1166.412, 54) :  17
(1166.412, 57) :  17
(1166.412, 82) :  17
(1166.515, 2) :  17
(1166.515, 10) :  17
(1166.515, 18) :  17
(1166.515, 29) :  17
(1166.515, 33) :  17
(1166.515, 52) :  17
(1166.515, 59) :  17
(1166.515, 62) :  17
(1166.515, 78) :  17
(1166.515, 79) :  17
(1166.515, 88) :  17
(1166.617, 19) :  17
(1166.617, 24) :  17
(1166.617, 31) :  17
(1166.617, 36) :  17
(1166.617, 38) :  1

(1306.047, 13) :  32
(1306.155, 44) :  32
(1306.155, 48) :  32
(1306.264, 63) :  32
(1306.589, 55) :  32
(1306.915, 23) :  32
(1306.915, 60) :  32
(1317.179, 99) :  33
(1317.288, 98) :  33
(1317.832, 65) :  33
(1317.832, 90) :  33
(1317.941, 42) :  33
(1318.159, 87) :  33
(1318.268, 91) :  33
(1318.548, 10) :  33
(1318.657, 6) :  33
(1318.657, 24) :  33
(1318.657, 52) :  33
(1318.766, 23) :  33
(1318.766, 32) :  33
(1318.766, 36) :  33
(1318.766, 68) :  33
(1318.766, 79) :  33
(1318.766, 100) :  33
(1318.875, 3) :  33
(1318.875, 8) :  33
(1318.875, 12) :  33
(1318.875, 31) :  33
(1318.875, 34) :  33
(1318.875, 47) :  33
(1318.875, 59) :  33
(1318.875, 61) :  33
(1318.875, 81) :  33
(1318.875, 88) :  33
(1318.965, 0) :  33
(1318.984, 2) :  33
(1318.984, 25) :  33
(1318.984, 35) :  33
(1318.984, 45) :  33
(1318.984, 48) :  33
(1318.984, 82) :  33
(1319.093, 27) :  33
(1319.093, 69) :  33
(1319.093, 73) :  33
(1319.093, 74) :  33
(1319.093, 80) :  33
(1319.093, 85) :  33
(1319.134, 95) : 

(1433.29, 51) :  42
(1433.29, 55) :  42
(1433.404, 4) :  42
(1433.404, 7) :  42
(1433.404, 22) :  42
(1433.404, 29) :  42
(1433.404, 34) :  42
(1433.404, 35) :  42
(1433.404, 59) :  42
(1433.404, 63) :  42
(1433.404, 71) :  42
(1433.517, 10) :  42
(1433.517, 38) :  42
(1433.517, 40) :  42
(1433.517, 53) :  42
(1433.517, 61) :  42
(1433.517, 80) :  42
(1433.631, 25) :  42
(1433.631, 27) :  42
(1433.631, 33) :  42
(1433.631, 46) :  42
(1433.631, 72) :  42
(1433.631, 81) :  42
(1433.659, 93) :  42
(1433.744, 12) :  42
(1433.744, 19) :  42
(1433.744, 26) :  42
(1433.744, 60) :  42
(1433.744, 69) :  42
(1433.744, 77) :  42
(1433.744, 82) :  42
(1433.858, 8) :  42
(1433.858, 31) :  42
(1433.858, 36) :  42
(1433.858, 45) :  42
(1433.858, 52) :  42
(1433.858, 62) :  42
(1433.858, 75) :  42
(1433.858, 88) :  42
(1433.887, 92) :  42
(1433.887, 94) :  42
(1433.972, 23) :  42
(1433.972, 74) :  42
(1433.972, 78) :  42
(1433.972, 84) :  42
(1434.114, 96) :  42
(1434.142, 100) :  42
(1434.199, 9) :  

(1562.657, 31) :  51
(1562.657, 33) :  51
(1562.657, 36) :  51
(1562.657, 53) :  51
(1562.657, 62) :  51
(1562.657, 63) :  51
(1562.657, 73) :  51
(1562.657, 75) :  51
(1562.657, 82) :  51
(1562.657, 84) :  51
(1562.657, 89) :  51
(1562.671, 94) :  51
(1562.671, 96) :  51
(1562.775, 9) :  51
(1562.775, 19) :  51
(1562.775, 46) :  51
(1562.775, 51) :  51
(1562.775, 54) :  51
(1562.775, 61) :  51
(1562.894, 10) :  51
(1562.894, 52) :  51
(1562.894, 60) :  51
(1562.894, 78) :  51
(1563.013, 26) :  51
(1563.013, 50) :  51
(1563.013, 55) :  51
(1563.131, 13) :  51
(1563.247, 101) :  51
(1563.497, 64) :  51
(1563.725, 6) :  51
(1583.264, 21) :  52
(1583.384, 20) :  52
(1583.384, 43) :  52
(1583.384, 99) :  52
(1583.623, 64) :  52
(1583.742, 65) :  52
(1583.742, 91) :  52
(1583.862, 86) :  52
(1583.862, 98) :  52
(1583.981, 42) :  52
(1583.981, 87) :  52
(1583.981, 90) :  52
(1584.204, 12) :  52
(1584.204, 35) :  52
(1584.204, 76) :  52
(1584.324, 13) :  52
(1584.324, 28) :  52
(1584.324, 37)

...

(17150.117, 11) :  1316
(17150.117, 41) :  1316
(17150.117, 47) :  1316
(17150.117, 52) :  1316
(17150.117, 54) :  1316
(17150.117, 61) :  1316
(17150.117, 73) :  1316
(17150.117, 83) :  1316
(17150.117, 89) :  1316
(17150.389, 86) :  1316
(17150.511, 49) :  1316
(17150.511, 50) :  1316
(17150.511, 63) :  1316
(17150.905, 13) :  1316
(17150.905, 34) :  1316
(17150.905, 62) :  1316
(17150.905, 84) :  1316
(17151.126, 97) :  1316
(17151.178, 91) :  1316
(17151.178, 99) :  1316
(17151.3, 40) :  1316
(17151.3, 66) :  1316
(17151.573, 65) :  1316
(17152.089, 8) :  1317
(17152.31, 92) :  1317
(17152.362, 42) :  1317
(17152.483, 15) :  1317
(17152.483, 59) :  1317
(17152.562, 100) :  1317
(17152.756, 98) :  1317
(17152.957, 101) :  1317
(17153.099, 93) :  1317
(17153.151, 20) :  1317
(17154.061, 5) :  1318
(17159.978, 6) :  1319
(17165.501, 38) :  1320
(17166.555, 1) :  1320
(17168.263, 18) :  1320
(17171.42, 39) :  1321
(17180.103, 75) :  1322
(17181.173, 86) :  1322
(17181.287, 2) :  1322
(

(17683.081, 83) :  1418
(17683.089, 87) :  1418
(17683.377, 92) :  1418
(17683.482, 14) :  1418
(17683.482, 77) :  1418
(17683.482, 89) :  1418
(17684.283, 12) :  1419
(17684.291, 64) :  1419
(17684.683, 11) :  1419
(17684.683, 57) :  1419
(17684.683, 60) :  1419
(17684.683, 71) :  1419
(17685.084, 25) :  1419
(17685.084, 27) :  1419
(17685.084, 33) :  1419
(17685.084, 49) :  1419
(17685.084, 51) :  1419
(17685.484, 34) :  1419
(17685.484, 73) :  1419
(17685.493, 43) :  1419
(17685.885, 8) :  1419
(17685.885, 50) :  1419
(17685.885, 58) :  1419
(17686.181, 94) :  1419
(17686.285, 28) :  1419
(17686.285, 48) :  1419
(17686.315, 1) :  1419
(17686.686, 5) :  1419
(17686.686, 19) :  1419
(17686.686, 75) :  1419
(17686.686, 81) :  1419
(17687.086, 4) :  1419
(17687.086, 6) :  1419
(17687.086, 10) :  1419
(17687.086, 15) :  1419
(17687.086, 41) :  1419
(17687.086, 47) :  1419
(17687.086, 52) :  1419
(17687.086, 61) :  1419
(17687.096, 99) :  1419
(17687.487, 13) :  1419
(17687.487, 46) :  14

(18305.247, 89) :  1540
(18305.654, 18) :  1540
(18306.062, 14) :  1540
(18306.062, 85) :  1540
(18307.284, 30) :  1541
(18307.284, 82) :  1541
(18307.447, 91) :  1541
(18307.692, 15) :  1541
(18307.692, 44) :  1541
(18307.854, 65) :  1541
(18308.08, 96) :  1541
(18308.099, 34) :  1541
(18308.507, 11) :  1541
(18308.507, 27) :  1541
(18308.507, 52) :  1541
(18308.914, 37) :  1541
(18308.914, 79) :  1541
(18309.322, 26) :  1541
(18309.322, 56) :  1541
(18309.322, 70) :  1541
(18309.73, 7) :  1541
(18309.73, 17) :  1541
(18309.73, 38) :  1541
(18309.73, 62) :  1541
(18309.893, 43) :  1541
(18310.137, 13) :  1541
(18310.137, 19) :  1541
(18310.952, 10) :  1542
(18310.952, 22) :  1542
(18311.36, 3) :  1543
(18311.36, 32) :  1543
(18311.36, 47) :  1543
(18311.36, 60) :  1543
(18311.749, 94) :  1543
(18311.767, 49) :  1543
(18311.767, 63) :  1543
(18311.931, 64) :  1543
(18312.175, 39) :  1543
(18312.175, 50) :  1543
(18312.175, 53) :  1543
(18312.564, 93) :  1543
(18312.583, 48) :  1543
(18

(18821.094, 12) :  1640
(18828.533, 8) :  1641
(18830.186, 18) :  1641
(18830.599, 66) :  1641
(18838.04, 24) :  1642
(18844.242, 2) :  1643
(18848.791, 31) :  1644
(18850.859, 28) :  1644
(18852.1, 88) :  1644
(18855.409, 35) :  1645
(18858.718, 6) :  1646
(18860.786, 25) :  1646
(18861.5, 43) :  1646
(18862.855, 74) :  1646
(18869.194, 100) :  1647
(18877.642, 87) :  1648
(18881.476, 30) :  1648
(18887.271, 15) :  1649
(18887.271, 28) :  1649
(18888.578, 93) :  1649
(18889.755, 33) :  1649
(18889.755, 78) :  1649
(18892.239, 74) :  1650
(18892.653, 82) :  1650
(18894.309, 19) :  1650
(18894.309, 58) :  1650
(18894.376, 97) :  1650
(18901.349, 79) :  1651
(18902.177, 70) :  1651
(18910.047, 31) :  1652
(18910.359, 64) :  1652
(18912.431, 98) :  1652
(18912.532, 16) :  1652
(18914.189, 49) :  1652
(18919.575, 25) :  1653
(18926.107, 21) :  1654
(18928.691, 19) :  1654
(18929.52, 51) :  1654
(18930.764, 26) :  1654
(18930.764, 58) :  1654
(18936.566, 10) :  1655
(18938.225, 45) :  1655


(19776.101, 20) :  1775
(19786.586, 79) :  1776
(19787.858, 31) :  1776
(19789.553, 32) :  1776
(19791.672, 46) :  1777
(19798.03, 78) :  1778
(19798.567, 43) :  1778
(19805.237, 25) :  1779
(19815.837, 53) :  1780
(19820.078, 50) :  1781
(19820.502, 16) :  1781
(19820.502, 68) :  1781
(19825.168, 28) :  1782
(19840.442, 78) :  1783
(19843.837, 57) :  1784
(19845.11, 5) :  1784
(19848.63, 98) :  1785
(19849.354, 45) :  1785
(19859.117, 68) :  1786
(19859.117, 80) :  1786
(19866.335, 31) :  1787
(19868.883, 78) :  1787
(19884.598, 39) :  1788
(19885.023, 38) :  1788
(19887.997, 36) :  1788
(19893.096, 27) :  1789
(19915.197, 83) :  1790
(19916.898, 2) :  1790
(19917.323, 25) :  1790
(19927.103, 37) :  1791
(19938.587, 51) :  1792
(19938.587, 88) :  1792
(19945.2, 97) :  1793
(19947.947, 54) :  1793
(19961.566, 14) :  1794
(19968.377, 69) :  1795
(19973.06, 16) :  1796
(19973.911, 15) :  1796
(19976.04, 48) :  1797
(19976.466, 79) :  1797
(19978.326, 65) :  1798
(19978.595, 46) :  1798
(